In [2]:
#!pip install torch
#!pip install docling_core
#!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.15.2
    Uninstalling typer-0.15.2:
      Successfully uninstalled typer-0.15.2


In [3]:
import torch
from docling_core.types.doc import DoclingDocument
from docling_core.types.doc.document import DocTagsDocument
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load images
image = load_image("/content/2.jpg")

# Initialize processor and model
processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")
model = AutoModelForVision2Seq.from_pretrained(
    "ds4sd/SmolDocling-256M-preview",
    torch_dtype=torch.bfloat16,
).to(DEVICE)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Convert this page to docling."}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=8192)
prompt_length = inputs.input_ids.shape[1]
trimmed_generated_ids = generated_ids[:, prompt_length:]
doctags = processor.batch_decode(
    trimmed_generated_ids,
    skip_special_tokens=False,
)[0].lstrip()

# Populate document
doctags_doc = DocTagsDocument.from_doctags_and_image_pairs([doctags], [image])
print(doctags)
# create a docling document
doc = DoclingDocument(name="Document")
doc.load_from_doctags(doctags_doc)

# export as any format
# HTML
# doc.save_as_html(output_file)
# MD
print(doc.export_to_markdown())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<doctag><otsl><loc_88><loc_98><loc_411><loc_151><ched>RS Synthetic Datasets<ched>Diversity<ched>Image Capture<ched>Assets<ched>Texture<ched>Task<ched># Images<nl><fcel>AICD [6]<fcel>City-Replica<fcel>Style-Extended<fcel>GSD (m)<fcel>Size<fcel>Perspective<fcel>Layout Geometry<fcel>Texture<nl><fcel>GTA-VSD [121]<fcel>X<fcel>*<fcel>800<fcel>Nadol,<fcel>Obl,<fcel>M<fcel>M<nl><fcel>Synthinel-1 [40]<fcel>0.3<fcel>0.3<fcel>572<fcel>572<fcel>Nad,<fcel>R<fcel>M+R<nl><fcel>Rafer'anles [161]<fcel>0.3<fcel>0.3<fcel>512<fcel>512<fcel>Nad,<fcel>R<fcel>M+R<nl><fcel>Syntic Cities [72]<fcel>0.1<fcel>0.3<fcel>1024<fcel>1024<fcel>Nad,<fcel>G<fcel>M+R<nl><fcel>GTAH [101]<fcel>X<fcel>*<fcel>1920<fcel>1920<fcel>Nadol,<fcel>G<fcel>M<nl><fcel>SynthWorld [83]<fcel>0.3<fcel>1.0<fcel>200<fcel>200<fcel>Nad,<fcel>P+<fcel>M<nl><fcel>SMARS [73]<fcel>0.3<fcel>1.0<fcel>210<fcel>210<fcel>Nad,<fcel>P+<fcel>M<nl><fcel>SynRS3D (Ours)<fcel>X<fcel>*<fcel>0.3<fcel>512<fcel>512<fcel>Nad,<fcel>R<nl><caption><loc_88><loc_50><lo

In [4]:
print(doc.export_to_markdown())

| RS Synthetic Datasets   | Diversity    | Image Capture   | Assets   | Texture   | Task        | # Images        |         |
|-------------------------|--------------|-----------------|----------|-----------|-------------|-----------------|---------|
| AICD [6]                | City-Replica | Style-Extended  | GSD (m)  | Size      | Perspective | Layout Geometry | Texture |
| GTA-VSD [121]           | X            | *               | 800      | Nadol,    | Obl,        | M               | M       |
| Synthinel-1 [40]        | 0.3          | 0.3             | 572      | 572       | Nad,        | R               | M+R     |
| Rafer'anles [161]       | 0.3          | 0.3             | 512      | 512       | Nad,        | R               | M+R     |
| Syntic Cities [72]      | 0.1          | 0.3             | 1024     | 1024      | Nad,        | G               | M+R     |
| GTAH [101]              | X            | *               | 1920     | 1920      | Nadol,      | G               | M 